In [1]:
# import matplotlib
# matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import cv2
import numpy as np
from scipy.optimize import minimize
from scipy.signal import find_peaks, peak_widths, argrelmax
from skimage.morphology import label, convex_hull_image
from scipy import ndimage as ndi
from tqdm import tqdm

In [2]:
# Bigger fontsize for x- and y-label
plt.rc('xtick', labelsize = 12)
plt.rc('ytick', labelsize = 16)

In [3]:
path_4x   = '/mnt/data/MAXIBONE/Goats/tomograms/hdf5-byte/scale/4x/'

path_1x = '/mnt/data/MAXIBONE/Goats/tomograms/hdf5-byte/scale/1x/'

seg_path  ='/mnt/data/MAXIBONE/Workspace/Mads/Concatenated_{scale}_samples/'.format(scale='1x')

mask_path ='/mnt/data/MAXIBONE/Workspace/Mads/Bit_masks/'

samples = ['769c.h5','770c.h5','771c.h5', '772.h5','774.h5','775.h5','778c.h5',\
          '780c.h5','785c.h5', '788c.h5','810c.h5','811.h5','815.h5','819.h5']

samples_1x = ['769.h5','769c.h5','770.h5', '770c.h5','771.h5', '771c.h5', '772.h5',\
              '774.h5','775.h5', '775c.h5','778.h5', '778c.h5','780c.h5', '785c.h5',\
              '788c.h5','810.h5','810c.h5','811.h5','815.h5','819.h5']

XX = np.linspace(1,255,254)

In [4]:
def img_show(x, n, cmap = None, vmax = None, vmin = None, dual_view = False, savefig = False):
    """Gives a quick view of an image, without writing too many lines"""
    """x: 2d-array to be projected; n: size of the imshow projection ; dual_view: If two images are to be shown simultaneuosly"""

    if dual_view:
        fig, ax = plt.subplots(ncols = 2, figsize = (n,n))
        im =  ax[0].imshow(x[0])
        im2 = ax[1].imshow(x[1])
        if savefig:
            fig.savefig('Fedt_dobbeltplot.eps')
    else:
        plt.figure(figsize = (n,n))
        plt.imshow(x,cmap = cmap, vmax = vmax, vmin = vmin)

In [5]:
# Creates indices round a circle with the given radius
def circle(r, center):
    """Creates indices in a circular pattern"""
    """r: Radius of the circle pattern; center: center of the circle"""
    T = int(2 * r * np.pi)
    x, y = r * [0], r * [0]
    for i,theta in enumerate(np.linspace(0,2*np.pi,r)):
        x[i] = r * np.cos(theta)
        y[i] = r * np.sin(theta)
    return (np.array(x)+center).astype(int), (np.array(y)+center).astype(int)


def rad_scan3d2(data):
    """Scan sample and creates histograms for each radial layer"""
    """data: 3d-array - CT data"""
    rad = data.shape[1]//2
    bins = np.arange(1,256)
    xy = [circle(r, rad) for r in range(1,rad)]
    arr = [np.histogram(np.concatenate((data[:,i[0],i[1]],data[:, i[0], -i[1] + rad*2])), bins = bins)[0] for i in xy]
    return np.array(arr)



def mask(data):
    """Mask for removing all values outside the cylinder """
    """data: 3d-array - CT data"""
    Ny, Nx = data.shape
    R = Nx / 2
    xs = np.linspace(-R,R,Nx)
    ys = np.linspace(-R,R,Ny)
    rs = np.sqrt(xs[None,:]**2 + ys[:,None]**2)
    mask = rs <= R
    return mask

def mask_seg3D(data,r1,r2):
    """Mask for removing values between two radii"""
    """data: 3d-array - CT data; r1: outer layer radius; r2: inner layers radius"""
    R = data.shape[1]/2 # the total radius of the CT
    l = np.zeros(data.shape[0])# empty array used for broadcasting from 2d to 3d
    xy = data.shape[1] # the diameter of the CT
    c = np.linspace(-R,R,xy)# an array with distances from the middle with as many step as the diameter
    rs = np.sqrt(c[None,:]**2 + c[:,None]**2)# broadcasting while calculating the euclidian distance from the center
    outer_mask = rs <= r1 # exclude values outside the outer ring
    inner_mask = rs >= r2 # exclude values inside the inner ring
    mask = outer_mask == inner_mask # where the inner and outer ring overlaps
    mask = l[:,None,None] + mask[None,:,:] # broadcasts the masked-array in 3d
    return mask.astype(bool) * data


# Scans sample along the z-, y- and x-axis
def zyx_scan2(data):
    """Takes raw CT input and creates histograms in Z-,Y- and X-direction"""
    """data: 3d-array - CT data"""
    masked = mask(data[0,:,:]) # uses the above mask function on the data 
    bins = np.arange(1,256)# bins used for histogram binning
    z_ref = np.array([data[z] * masked for z in range(np.min(data.shape))])# applies the mask onto every layer
    
    #Creates thte 2d histograms for Z, Y and X with list comprehension
    z_list = np.array([np.histogram(data[z] * masked,         bins = bins)[0] for z in range(data.shape[0])])
    y_list = np.array([np.histogram(z_ref[:,y,:].reshape(-1), bins = bins)[0] for y in range(data.shape[1])])
    x_list = np.array([np.histogram(z_ref[:,:,x].reshape(-1), bins = bins)[0] for x in range(data.shape[2])])
    return z_list, y_list, x_list

In [6]:
def implant_bitmask(data,threshold):
    """Creates a mask for implant and air for each layer, combining them to a full bit-maks"""
    """data: 3d-array - CT data; threshold: Intensity threshold to exclude all data below the given value """
    implant_mask = np.zeros_like(data, dtype = bool)# empty array for mask
    implant_mask[data > threshold] = True # marks everything equal or above threshold as True
    tot = np.prod(data.shape[1:])//100 # 1 percent of the total amount of voxels in one slice
    for i in range(len(implant_mask)):# for loop for labeling clusters
        lab = ndi.label(implant_mask[i])# labels clusters
        test = np.where(lab[0] > 0)# checks how many voxels are above threshold 
        clusters = np.array([np.where(lab[0]==x)[0].size for x in range(1,5)])# caculate sizes of the labeled clusters
        if len(test[0]) < tot:# if the number of voxels above threshold is less than 1 percent, then everything is excluded
            implant_mask[i] = False
        else:
            max_cluster = clusters.argmax() + 1 # the biggest cluster that is not 0 i.e empty voxels below threshhold
            max_indices = np.where(lab[0] == max_cluster)
            y,x = max_indices
            left =  y[x.argmin()] # max y val at min x val
            right = y[x.argmax()] # max y val at max x val
            # The implant is not always level in the horizontal axis, so this if statement find out which sides is higher
            # to mark the areas above properly
            # Then marks everything above as True and fill holes, leaving the final mask
            if left >= right:
                implant_mask[i, :right+2] = True
                implant_mask[i, right:left+2, :x.min()] = True
                implant_mask[i] = ndi.binary_fill_holes(implant_mask[i])
            elif left <= right:
                implant_mask[i, :left+2] = True
                implant_mask[i, left:right+2, x.max():] = True
                implant_mask[i] = ndi.binary_fill_holes(implant_mask[i])
    return ~implant_mask # inverts mask in the end

def top_mask_fixer(mask,idx):
    """Fixed error of the implant_bitmask"""
    """mask: mask to be applied; idx: indices of the mask"""
    if idx.min() < 150:
        for i in range(int(idx.min()),int(idx.max())):
            if np.all(mask[i]):
                mask[i] = False
            else:
                continue
    return mask
    
def mask_multiplier(tup,idx):
    """Combined bit-masks, or returned 1 if no mask were given"""
    """tup: the tuple of masks or integer or float; idx: indices of the mask"""
    if type(tup) == type((1,2,2)):
        mask = top_mask_fixer(tup[0]['mask_voxels'][idx],idx)
        for i in range(1,len(tup)):
            mask *= tup[i]['mask_voxels'][idx]
        return mask
    elif type(tup) == type(1) or type(tup) == type(1.0):
        return 1
    else:
        return top_mask_fixer(tup['mask_voxels'][idx], idx)

In [7]:
# Loads data and uses all above functions to create four distinct 2D-histograms w.r.t. x-,y,z- and radial axis
# Includes masks for removing unwanted data 
def load_2D_data(data, chunk_size, rotate, masks):
    """Takes CT-data and creates 2d histograms for radial-, Z-, Y- and X-axis"""
    """data: 3d-array - CT data; chunk_size: number of layers to be scanned simultaneuously """
    """rotate: True if samples needs to be rotated, else False; masks: the masks for excluding unnecessary data, integer if no mask"""
    z_len  = data['voxels'].shape[0]# How many layers are in the Z-direction
    y_len  = data['voxels'].shape[1]# How many layers are in the Y- and X-direction
    rad_len = y_len//2 # the radius of the sample 
    hist_z =   np.zeros((z_len,       254))# histogram of the 4 different transformations
    hist_y =   np.zeros((y_len,       254))
    hist_x =   np.zeros((y_len,       254))
    hist_rad = np.zeros((rad_len - 1, 254))

    for i in tqdm(range(0,z_len, chunk_size)):
        ni = min(chunk_size, z_len-i)#takes minimum of the chunk-size and the remaining layers to make sure all layer are included
        
        indices = np.arange(i, i+ni)# Indices of the chunk
        sample_frac = data['voxels'][indices]# the chunk of the Ct is loaded
        
        if rotate:# if the sample needs to be rotated
            sample_frac = np.rot90(sample_frac, 3, (1,2))
        
        
        sample_frac = sample_frac * mask_multiplier(masks,indices)# add masks to the chunk
        
        R = rad_scan3d2(sample_frac)# Histogram for radial
        Z,Y,X = zyx_scan2(sample_frac)# Histogram for Z,Y and X
        
        hist_rad += R# Adds histograms to respective array
        hist_y   += Y
        hist_x   += X
        hist_z[indices]   += Z
    print('Data has been loaded!')

    return hist_rad, hist_z, hist_y, hist_x

def load_2D_correc(data, chunk_size, rotate, masks):
    """Same as load_2D_data but only for radial"""
    z_len  = data['voxels'].shape[0]
    y_len  = data['voxels'].shape[1]
    rad_len = y_len//2
    hist_rad = np.zeros((rad_len - 1, 254))

    for i in tqdm(range(0,z_len, chunk_size)):
        ni = min(chunk_size, z_len-i)
        
        indices = np.arange(i, i+ni)
        sample_frac = data['voxels'][indices]
        
        sample_frac = sample_frac * mask_multiplier(masks,indices)
        
        R = rad_scan3d2(sample_frac)
        hist_rad += R
    print('Data has been loaded!')

    return hist_rad

In [8]:
def save_fits(hist, axis_name, sample_name, scale, mask):
    """Saves a fitted 2d histogram as a .npy with proper naming"""
    """hist: 2d histogram; axis_name: axis of the histogram i.e. ZYX or radial"""
    """sample_name: Name of the CT; scale: What scale the sample is used i.e. 1x,2x,4x,8x and 16x"""
    if mask == 1: # if only one mask is applied
        string = '_masked'
        np.save(axis_name + '_fits_' + sample_name[:-3] +'_'+ scale + string, hist)
    elif mask == 2:# two masks applied
        string = '_double_masked'
        np.save(axis_name + '_fits_' + sample_name[:-3] +'_'+ scale + string, hist)
    elif mask == 0:# no mask applied
        sn = sample_name
        np.save(axis_name + '_fits_' + sn[:-3]+'_' + scale, hist)

In [9]:
def save_vars(hist, axis_name, sample_name, scale, mask):
    """Saves the variables of the distributions in each layer as a .npy files"""
    """Input are the same as in save_fits"""
    if mask==1:
        string = '_masked'
        np.save(axis_name + '_parameters_' + sample_name[:-3] +'_'+ scale + string, hist)
    elif mask== 0:
        sn = sample_name
        np.save(axis_name + '_parameters_' + sn[:-3]+'_' + scale, hist)
    elif mask==2:
        string = '_double_masked'
        np.save(axis_name + '_parameters_' + sample_name[:-3] +'_'+ scale + string, hist)
        

def load_processed_histograms_rzyx(sample,scale,mask):
    """Loads the histograms of a given sample"""
    if mask ==1:
        return np.load('rad_hist_'+ sample[:-3]+'_'+ scale +'_masked.npy', allow_pickle = True ),\
    np.load('z_hist_'+ sample[:-3]+'_'+ scale +'_masked.npy', allow_pickle = True ),\
    np.load('y_hist_'+ sample[:-3]+'_'+ scale +'_masked.npy', allow_pickle = True ),\
    np.load('x_hist_'+ sample[:-3]+'_'+ scale +'_masked.npy', allow_pickle = True )
    elif mask==0:
        return np.load('rad_hist_'+ sample[:-3]+'_'+ scale +'.npy', allow_pickle = True ),\
    np.load('z_hist_'+ sample[:-3]+'_'+ scale +'.npy', allow_pickle = True ),\
    np.load('y_hist_'+ sample[:-3]+'_'+ scale +'.npy', allow_pickle = True ),\
    np.load('x_hist_'+ sample[:-3]+'_'+ scale +'.npy', allow_pickle = True )
    elif mask ==2:
        return np.load('rad_hist_'+ sample[:-3]+'_'+ scale +'_double_masked.npy', allow_pickle = True ),\
    np.load('z_hist_'+ sample[:-3]+'_'+ scale +'_double_masked.npy', allow_pickle = True ),\
    np.load('y_hist_'+ sample[:-3]+'_'+ scale +'_double_masked.npy', allow_pickle = True ),\
    np.load('x_hist_'+ sample[:-3]+'_'+ scale +'_double_masked.npy', allow_pickle = True )
    

In [10]:
def load_fits(sample, scale, path = None):
    """Function that loads the fitted histograms of a given sample"""
    return np.load('rad_fits_'+ scale + '_' + sample[:-3] +'_double_masked.npy', allow_pickle = True ),\
    np.load('z_fits_'  + scale + '_' + sample[:-3] +'_double_masked.npy', allow_pickle = True ),\
    np.load('y_fits_'  + scale + '_' + sample[:-3] +'_double_masked.npy', allow_pickle = True ),\
    np.load('x_fits_'  + scale + '_' + sample[:-3] +'_double_masked.npy', allow_pickle = True )
    
def load_parameters(sample, scale, path = None):
    """Function that loads the fitted parameters of a given sample"""
    return np.load('rad_parameters_'+ scale + '_' + sample[:-3] +'.npy', allow_pickle = True ),\
    np.load('z_parameters_'  + scale + '_' + sample[:-3] +'.npy', allow_pickle = True ),\
    np.load('y_parameters_'  + scale + '_' + sample[:-3] +'.npy', allow_pickle = True ),\
    np.load('x_parameters_'  + scale + '_' + sample[:-3] +'.npy', allow_pickle = True )

In [11]:
def save_hist(hist, axis_name, sample_name,scale, mask):
    " Function that saves a histogram as .npy file"
    if mask == 1:# if one mask is applied
        string = '_masked'
        np.save(axis_name + '_hist_' + sample_name[:-3] +'_'+ scale + string, hist)
    
    elif mask == 2:# if two masks are applied 
        string = '_double_masked'
        np.save(axis_name + '_hist_' + sample_name[:-3] +'_'+ scale + string, hist)
        
    elif mask== 0:# if no mask are applied
        sn = sample_name
        np.save(axis_name + '_hist_' + sn[:-3]+'_' + scale, hist)

def save_labels(hist, axis_name, sample_name,scale, mask):
    """Fucntion that save the labels of the the fitted histograms"""
    if mask == 1:
        string = '_masked'
        np.save(axis_name + '_labels_' + sample_name[:-3] +'_'+ scale + string, hist)
    
    elif mask == 2:
        string = '_double_masked'
        np.save(axis_name + '_labels_' + sample_name[:-3] +'_'+ scale + string, hist)
        
    elif mask== 0:
        sn = sample_name
        np.save(axis_name + '_labels_' + sn[:-3]+'_' + scale, hist)
        
def load_n_save_hists(sample, scale, masks, rotate, masked, concat):
    """Function that loads and saves the histograms"""
    if concat:
        path = '/mnt/data/MAXIBONE/Workspace/Mads/Concatenated_'+ scale +'_samples/' + scale + '_'+ sample[:-3]+'_concat.h5'
    else:
        path = '/mnt/data/MAXIBONE/Goats/tomograms/hdf5-byte/scale/'+ scale +'/' + sample
    file = h5py.File(path, 'r')
    n = file['voxels'].shape[0]//(10//int(scale[0]))
    print(n)
    R,Z,Y,X = load_2D_data(file, n, rotate, masks)
    save_hist(R,'rad', str(sample), scale, masked)
    save_hist(Y,'y'  , str(sample), scale, masked)
    save_hist(X,'x'  , str(sample), scale, masked)
    save_hist(Z,'z'  , str(sample), scale, masked)

In [12]:
def save_fits_rzyx(fitted_distributions, variables, scale, sample, mask):
    """Saves all fitted distributions and their respective variables to a .npy file"""
    R = fitted_distributions[0]
    Z = fitted_distributions[1]
    Y = fitted_distributions[2]
    X = fitted_distributions[3]
    R_vars = variables[0]
    Z_vars = variables[1]
    Y_vars = variables[2]
    X_vars = variables[3]
    if mask ==0:
        string = ''
    elif mask ==1:
        string = '_masked'
    elif mask ==2:
        string = '_double_masked'
        
    np.save('rad_fits_' + scale  + '_' + sample[:-3] + string, R)
    np.save('z_fits_'   + scale  + '_' + sample[:-3] + string, Z)
    np.save('y_fits_'   + scale  + '_' + sample[:-3] + string, Y)
    np.save('x_fits_'   + scale  + '_' + sample[:-3] + string, X)
    
    np.save('rad_parameters_' + scale + '_' + sample[:-3] + string, R_vars)
    np.save('z_parameters_'   + scale + '_' + sample[:-3] + string, Z_vars)
    np.save('y_parameters_'   + scale + '_' + sample[:-3] + string, Y_vars)
    np.save('x_parameters_'   + scale + '_' + sample[:-3] + string, X_vars)
    

In [13]:
### Generel functions for fitting and describing data

# Baseline distribution for all fits of histograms
def power(vars,xs):
    """The function for the generalized gaussion distribution (GGD)"""
    n = vars.size//4
    abcd = np.reshape(vars,(4 ,n)).T
    # broadcasts all sets of parameters in their own subsets so each distribution are separated
    A, B, C, D = abcd[:,0,np.newaxis]**2, abcd[:,1,np.newaxis]**2, abcd[:,2,np.newaxis], abcd[:,3,np.newaxis]**2
    X          = xs[np.newaxis,:]
    d = 1 + np.sqrt(1 / (1 + D)) # Interval between 1 and 2
    return np.sqrt(A) * np.exp(- np.sqrt(B) * np.power(np.abs(X - C), d ))


def neg(f):
    """Penalty function to ensure fit doesn't go outside curves, a.e. so the distribution doesn't describe something there isn't there"""
    neg = (np.abs(f) - f)/2
    return neg


def error_power(vars,x_data,f_exact, overshoot_penalty):
    """The fucntion the minimizer evaluates the variables for the GGDs function"""
    fopt = np.sum(power(vars,x_data), axis = 0)
    E    = np.sum((f_exact - fopt)**2)  + overshoot_penalty * np.sum(neg(f_exact - fopt)**2)
    return E

def sigma_spread(x,y,peaks):
    """Reverts the histogram to individual data points to find intial spread for the distributions"""
    left = np.floor(peaks[1]['left_ips']).astype(int)# left side of the peaks found
    right = np.ceil(peaks[1]['right_ips']).astype(int) # right side of the peaks found
    ips = np.vstack((left,right)).T # stack left and right tails into single array
    ips_expanded = [np.arange(i.min(),i.max()+1, 1) for i in ips] # indices in between left and right tails
    values = [np.repeat(x[j],y[j].astype(int)) for j in ips_expanded] # the values given by the axis
    spread = np.array([np.std(values[k]) for k in range(len(values))]) # calculates the spread of each peak
    if np.any(spread==0):
        idx = np.where(spread==0)[0]
        spread[idx] = 0.1
        return spread
    else:
        return 1 / (2*spread)**2


#def normalise(x):
#    """normalises wrt maximum values in the slice"""
#    if (np.max(x)-np.min(x)) == 0:
#        return np.zeros_like(x)
#    else:
#        return(x-np.min(x))/(np.max(x)-np.min(x))
    
def normalise(x):
    """normalises distributions wrt to number of voxels pr slice"""
    if (np.max(x)-np.min(x)) == 0:
        return np.zeros_like(x)
    else:
        return x/x.size

In [14]:
def sigma_spread2(x,y):
    """Function that avoids error while finding the spread"""
    values = np.repeat(x,y.astype(int))
    spread = np.std(values)
    if np.isnan(spread): # if spread is zero
        return 0 # to avoid True-divide
    else:
        return spread
    
def error_layer(distr):
    """find the standard deviation of each distribution in a class"""
    error_arr = np.zeros((distr.shape[0],distr.shape[1]))
    for i in range(len(error_arr)):
        error_arr[i] += np.array([sigma_spread2(XX,distr[i][x]) for x in range(len(new_fit))])
    return error_arr

def convert_d(vars):
    """Reconvert d values back to actual values for saving variables"""
    vars[-1] = 1 + np.sqrt(1 / (1 + vars[-1]))
    return vars

In [15]:
# Upgraded minimizer
def bounds(x0):
    """Make bounds for the initial guess for the distribution before being minimized"""
    n = x0.size//4
    A = np.array([(i/2,i) for i in x0[:n]])
    B = np.array([(0.2,5) for i in range(n)])
    C = np.array([(i-1,i+1) for i in x0[2*n:3*n]])
    D = np.array([(0,1000) for i in range(n)])
    return np.concatenate((A,B,C,D))

def peak_info_yseg(data):
    """Function that find peak positions"""
    initial_peaks = find_peaks(data)[0]# locate all "peaks" 

    PW,PH,l_ips,r_ips = peak_widths(data, initial_peaks)# calculate the peak tails
    MH = np.mean(PH)/5# height threshold for accepting peaks as actual peaks
    final = np.array([x for x in range(len(PW)) if PH[x] >= MH]).astype(int) # Filters actual peaks from "peaks"
    dict  = {'left_ips': l_ips[final],'right_ips': r_ips[final]}
    return initial_peaks[final], dict

def initial_trial6(xs,data, method = 'power'):
    """ returns the initial guess values for the minimizer """
    init_peaks = peak_info_yseg(data)

    peak_idx = init_peaks[0]
    n = len(peak_idx) # number of peaks
    A = data[peak_idx] # Amplitude of the distribution
    C = xs[peak_idx] # Mean/mode of distribution
    B = sigma_spread(xs,data,init_peaks) # the spread of each distribution
    if method == 'gauss':
        vars = np.concatenate((A,B,C))
    elif method == 'power':
        D = np.ones(n)*0.5
        vars = np.concatenate((A,B,C,D))
    return vars


def minimizer_mask(xs, data, penalty):
    """return minimized-parameters and -distributions"""
    vars = initial_trial6(xs, data)
    
    if len(vars) < 1: # return zeros if no peaksare registered
        return np.zeros(1), np.zeros(len(xs))
    else:# creates bounds and minimizes, if peaks are registered
        bnds = bounds(vars)
        fit = minimize(error_power,vars, args = (xs, data, penalty), bounds = bnds)
        return fit.x, power(fit.x, xs)

In [16]:
def mask_seg3Deux(data,r1,r2):
    """return mask in 3d, repeat function"""
    R = data.shape[1]/2
    l = np.zeros(data.shape[0])
    xy = data.shape[1]
    c = np.linspace(-R,R,xy)
    rs = np.sqrt(c[None,:]**2 + c[:,None]**2)
    outer_mask = rs <= r1
    inner_mask = rs >= r2
    mask = outer_mask == inner_mask
    if data.shape == 3:
        mask = l[:,None,None] + mask[None,:,:]
        return np.where(mask==True)
    else:
        return np.where(mask==True)

In [17]:
def label_loc(image,clus_frac, plot):
    """Function that filter off clusters below a treshhold"""
    """image: 2d array; clus_frac: percentage threshold; plot: return imshow it True """
    counts = np.bincount(image.flatten()) # counts the number squares in each cluster
    cluster_size = np.sum(counts[1:]) * (clus_frac/100) # To exclude clusters containing less than 1 % of the segmented data
    rang = np.where(counts > cluster_size)[0] # list of clusters bigger than desired cluster size
    list = []
    for i in range(1,len(rang)): # not sure if this is necessary ?
        picf = np.copy(image)
        picf[np.where(picf != rang[i])] = 0 # converts values indifferent to cluster integer to 0, so only that cluster is extracted
        list.append(picf//rang[i]) # Appends single cluster to list
        if plot:
            plt.figure(figsize  = (10,10))
            plt.imshow(picf)
    arr = np.asarray(list, dtype = int)
    for i in range(len(arr)):
        arr[i] *=(i+1)
    return arr


In [18]:
def list_2_arr(list):
    """Convert the python lists with distributions into single 2d-array"""
    y = len(list)
    x = list[0].shape[-1]
    array = np.zeros((y,x))
    for i in range(y):
        for j in range(len(list[i])):
            array[i,:] += list[i][j]
    return array

def class_tracker(hist,vars):
    """Creates binary array with 1 on positions of mean/mode of peaks"""
    arr = np.zeros_like(hist, dtype = 'float32')
    for i in range(len(arr)):
        n = vars[i].size//4
        cs = vars[i][2*n:3*n]
        cs = cs.astype(int)
        arr[i,cs] += 1
    return arr


In [19]:
def rad_fitter(data, penalty):
    """Returns list of parameters and distributions for radial"""
    hist_rad = data
    rad_list = []
    rad_vars = []
    for i in tqdm(range(len(hist_rad))):
        fee = minimizer_mask(XX,hist_rad[i],penalty)
        rad_vars.append(fee[0])
        rad_list.append(fee[1])
        #if i%(len(hist_rad)//10) ==0:
            #print(np.round((100*i)/len(hist_rad)),' %  done')
    print('The radial-histogram has been fitted')
    return rad_list, rad_vars


def y_fitter(data, penalty):
    """Returns list of parameters and distributions for Y"""
    hist_y = data
    y_list = []
    y_vars = []
    for i in tqdm(range(len(hist_y))):
        fee = minimizer_mask(XX,hist_y[i],penalty)
        y_vars.append(fee[0])
        y_list.append(fee[1])
        #if i%(len(hist_y)//10) ==0:
            #print(np.round((100*i)//len(hist_y)), ' %  done')

    print('The y-histogram has been fitted')
    return y_list, y_vars

def z_fitter(data, penalty):
    """Returns list of parameters and distributions for Z"""
    hist_z = data
    z_list = []
    z_vars = []
    for i in tqdm(range(len(hist_z))):
        fee = minimizer_mask(XX,hist_z[i],penalty)
        z_vars.append(fee[0])
        z_list.append(fee[1])
        #if i%(len(hist_z)//10) ==0:
            #print(np.round((100*i)/len(hist_z)),' %  done')
    print('The z-histogram has been fitted')
    return z_list, z_vars

def x_fitter(data, penalty):
    """Returns list of parameters and distributions for X"""
    hist_x = data
    print(len(data))
    x_list = []
    x_vars = []
    for i in tqdm(range(len(hist_x))):
        fee = minimizer_mask(XX,hist_x[i],1)
        x_vars.append(fee[0])
        x_list.append(fee[1])
        #if i%(len(hist_x)//10) ==0:
            #print(np.round((100*i)/len(hist_x)), ' %  done')
    print('The x-histogram has been fitted')
    return x_list, x_vars

def combined_autofitter(data, penalty):
    """combined Fitting function for radial-, Z-,Y-,X-histograms"""
    r_fit, r_var = rad_fitter(data[0], penalty)
    z_fit, z_var = z_fitter  (data[1], penalty)
    y_fit, y_var = y_fitter  (data[2], penalty)
    x_fit, x_var = x_fitter  (data[3], penalty)
    rad = [r_fit, r_var]
    z =   [z_fit, z_var]
    y =   [y_fit, y_var]
    x =   [x_fit, x_var]
    return rad, z, y, x 

In [20]:
# Some peaks were underdescribed, these functions tried to compensate for that 
def insert_missing_peaks(data,params,indices):
    """update the parameters by adding more to each array"""
    n = params.size//4
    m = len(indices)
    a = data[indices]
    c = indices
    b = np.ones_like(indices)
    d = b*0.5
    new_values = np.concatenate((a,b,c,d))
    place = np.concatenate((m*[0],m*[n],m*[2*n], m*[3*n]))
    new_params = np.insert(params,place, new_values)
    return new_params

def find_missing_peaks(data,fit):
    """Located all peaks to insert """
    mp_list = []
    fit = list_2_arr(fit)
    diff = data - fit
    for i in range(len(fit)):
        dib = argrelmax(diff[i])[0]
        id1 = dib > 44
        id2 = dib < 75
        ID = id1==id2
        diffp = (diff[i]-fit[i])[dib[ID]]
        if len(diffp[diffp>8000]) >=1 :
            mp_list.append([i,dib[ID][diffp>8000]])
    return np.array(mp_list).reshape(len(mp_list),2)

def improve_fits(xs, data, fit, penalty):
    """Applies missing peaks to the 2d histogram"""
    hidden_peaks = find_missing_peaks(data,fit[0])
    closed_fit = np.copy(fit[0])
    updated_params = np.copy(fit[1])
    print(len(hidden_peaks))
    for i,j in tqdm(hidden_peaks):
        new_params = insert_missing_peaks(data[i],fit[1][i],j)
        bnds = bounds(new_params)
        n_fit = minimize(error_power,new_params, args = (xs, data[i], penalty), bounds = bnds)
        updated_params[i] = new_params
        closed_fit[i] = power(n_fit.x,XX)
        #function better 
    return closed_fit, updated_params
 

In [21]:
def find_ridges(fits, k = 0, label_frac = 5 , e_iter = 1 , d_iter = 6, show = True):
    """ Find ridges from the list of distributions"""
    """fits: list of distributions and parameters; k: number of ones in middle column of 3x3 kernel"""
    """label_frac: percentage threshhold; e_iter: number of erosions; d_iter: number of dilations; show: plot image if True"""
    distr  = list_2_arr(fits[0])# 2d array from list of distributions
    ridges = class_tracker(distr, fits[1])# binary array of all peak positions
    kern = np.zeros((3,3), dtype = 'uint8')# 3x3 kernel
    kern[k:3,1] += 1# add ones to middle column, k =0, fills entire colums, k=1 only 2/3 of column is filled
    erode = cv2.erode (ridges,  kernel = kern, iterations = e_iter)#erodes
    d2    = cv2.dilate(erode,   kernel = kern, iterations = d_iter)# dilates

    lab = label(d2, connectivity = 2)# ndimage label: Labels clusters with integers
    lab_loc = label_loc(lab, label_frac, False)# removes clusters below threshhold sizes
    print(len(lab_loc),' labels')# prints the number of clusters/labels
    if show:# if True, plots labels and fitted 2d hist next to each other
        img_show((lab_loc.sum(axis=0),distr), 25, dual_view = True)
    return lab_loc



def distr_sorter(distr,labels):
    """Sorting distributions into classes"""
    """distr: list of distributions; labels: labels given from find_ridges function"""
    Arr = np.zeros((len(labels),len(distr),distr[0].shape[-1]))
    print('Shape of array:', Arr.shape)# print shape of label array. axis 0 is number of labels

    for i in range(1,len(distr)-1):
        idxy = np.where(labels[:,i]>0)# where in the 2d array labels are situated
        if distr[i].sum() == 0.0 or distr[i].sum() == 0:# to avoid error 
            continue
            
        elif len(distr[i]) > 1 and len(idxy[1]) > 1:
            args = distr[i].argmax(axis=1)# where peaks are situated
            idxy = np.where(labels[:,i] > 0)# redo idxy for some reason
            labs = [abs(idxy[1]-x).argmin() for x in args]# combares all peaks positions with labels to determine which class is closest in value
            lad = idxy[0][labs]# indices for matching labels
            for j in range(len(distr[i])):
                Arr[lad[j],i,:] += distr[i][j]# appends distributions to matching label
                
        else:# if only one labels is in the layer
            for j in range(distr[i].shape[0]):
                #print(distr[i].shape[0])
                Arr[idxy[0],i] += distr[i][j]# adds distribution to label
    return Arr

# Sorts parameters for the distributions into appropiate classes

def parameter_sorter(params,labels):
    """Takes the lists of parameters from each layer, sorting them into class"""
    """If the class isn't present in the given layer, a 0 is placed instead"""
    """params: list of parameters from minimizer; labels: labels gained from find_ridgees function"""
    print('Size of array',len(params),'- Number of labels',len(labels))# print size of array and number of labels

    mad_list = []
    for f in range(len(labels)):
        mad_list.append([])

    for i in range(len(params)):
        N = len(params[i])//4
        cs = params[i][2*N:3*N]# peak positions
        idxy = np.where(labels[:,i] > 0)# where the labeled array assigns a class
        if len(idxy[0]) < 1:
            lad = None
        else:
            labs = [abs(idxy[1]-x).argmin() for x in cs] # Finds overlap between class and distribution
            lad = idxy[0][labs] # applies indices to the overlapping distribution
        empty_class = np.isin(np.arange(len(labels)),lad) # array for classes the overlap and doesn't
        empty = np.arange(len(labels))[~empty_class]# indices for non overlap
        full  = np.arange(len(labels))[empty_class] # indices for overlap

        for k in empty:
            mad_list[k].append(0)
        for j in full:
            idx = np.where(lad==j)[0]
            mad_list[j].append([convert_d(params[i][y::N]) for y in idx])
    return mad_list

In [22]:
# no longer used
def combined_fit_improver(data,fits):
    """Combined the three funtions for improving fits"""
    upd_r = improve_fits(XX,data[0], fits[0], 5)
    print('rad_fits has been updated')
    upd_z = improve_fits(XX,data[1], fits[1], 5)
    print('z_fits has been updated')
    upd_y = improve_fits(XX,data[2], fits[2], 5)
    print('y_fits has been updated')
    upd_x = improve_fits(XX,data[3], fits[3], 5)
    print('x_fits has been updated')
    return  upd_r, upd_z, upd_y, upd_x

In [23]:
def create_radmask(data,rmin):
    """Creates a mask for each layer in the radial transformation"""
    """data: CT-data; rmin: number of layers from 0 to rmin acting as 'first' layer"""
    inner_layer = mask_seg3Deux(data,rmin,0)# number of voxels scale with r, so the first layer are from 0:rmin
    remaining_layers = np.array([mask_seg3Deux(data,r,r-1) for r in range(rmin,(data.shape[1]//2))])# creates masks for remaming layers
    #one mask for each +1 increment of the radius
    mask = [inner_layer]# list with inner layer indices 
    for i in range(len(remaining_layers)):# appends remaining indices to list
        mask.append(remaining_layers[i])
    return np.array(mask)

In [24]:
def create_h5py_files(shape, axis, scale, sample, n_class):
    """create h5py files to compress new segmented CT's"""
    """axis:Z,Y,X or Radial; scale:1x,2x,4x... ; sample: .h5 CT-sample name; n_class: number of classes"""
    output_files = [h5py.File(axis +'_segmentation_' + scale + '_' + sample[:-3] +'_'+ 'Class'+ str(x+1) + '.h5','w') for x in range(n_class)]
    for i in range(n_class):
        output_files[i].create_dataset("mask_voxels", shape, dtype=np.float, compression="lzf")
    output_voxels = [output_files[v]['mask_voxels'] for v in range(n_class)]
    return output_voxels, output_files

In [25]:
# Segmentation functions 

def zyx_test_segmentation(data, fit, zyx, depth):
    """Segmentation function for Z, Y and X"""
    arrs = []
    
    if zyx == 0: # meaning Z-segmentation
        for j in range(len(fit)):
                arrs.append(np.array([normalise(fit[j][val+depth,data[val,:,:]]) for val in range(data.shape[zyx])]))
    elif zyx == 1: # Y-Segmentation
        for j in range(len(fit)):
            arrs.append(np.array([normalise(fit[j][val, data[:,val,:]]) for val in range(data.shape[zyx])]))
                
    elif zyx == 2: # X-Segmentation
        for j in range(len(fit)):
            arrs.append(np.array([normalise(fit[j][val, data[:,:,val]]) for val in range(data.shape[zyx])]))
                
    return arrs

In [26]:
# Segmentation functions 

def zyx_stodder_segmentation(data, fit, zyx, depth):

    arrs = []
    
    if zyx == 0: # meaning Z-segmentation
        for j in range(len(fit)):
            if len(data) %100==0:
                arrs.append(np.array([normalise(fit[j][val+depth,data[val,:,:]]) for val in range(data.shape[zyx])]))
            else:
                arrs.append(np.array([normalise(fit[j][val+depth,data[val,:,:]]) for val in range(data.shape[zyx]-1)]))
                
    elif zyx == 1: # Y-Segmentation
        for j in range(len(fit)):
            arrs.append(np.array([normalise(fit[j][val, data[:,val,:]]) for val in range(data.shape[zyx])]))
                
    elif zyx == 2: # X-Segmentation
        for j in range(len(fit)):
            arrs.append(np.array([normalise(fit[j][val, data[:,:,val]]) for val in range(data.shape[zyx])]))
                
    return arrs

In [27]:
def resin_mask(sample, scale, z_fit, chunk_size, concat, rot):
    """Creates resin-mask"""
    """sample: .h5 file of the CT; scale: the scale of the data i.e 1x, 2x etc"""
    """z_fit: The distribution to segment with; chunk_size: size of CT chunks used for each iteration """
    """concat: if the CT to be used has been volume matched or not; rot: if the sample are to be rotated"""
    if concat:
        tomogram     = h5py.File(seg_path + scale + '_' + sample[:-3] + '_concat.h5', 'r')
    else:
        tomogram     = h5py.File(path_1x + sample, 'r')
    
    implant_mask = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + sample, 'r')# implant bitmask to remove implant and air
    fit = z_fit.reshape(1,z_fit.shape[0],z_fit.shape[1])# shape the arr, so it matches the required shape for the segmentation
    
    shape = implant_mask['mask_voxels'].shape# to get dimensions to know the number of iterations through the z-axis

    
    resin_file = h5py.File('Resin-bitmask_' + scale + '_' + sample,'w')# create the resin bit-mask file
    resin_file.create_dataset("mask_voxels", shape, dtype=np.bool, compression="lzf")# create dataset to input data
    output_voxels = resin_file['mask_voxels']# variable to input segmented data into

    print('H5py-files are ready to be written')

    for i in tqdm(range(0, shape[0], chunk_size)): #shape[zyx]
        ni = min(chunk_size, shape[0]-i) #zyx determines which axis to be iterated through
        if rot:# if the chunk are to be rotated
            Layer = np.rot90(tomogram['voxels'][i:i+ni],3,(1,2))# loads chunk of the CT
        else:
            Layer = tomogram['voxels'][i:i+ni]# loads chunk of the CT
            
        masked_sample = Layer * implant_mask['mask_voxels'][i:i+ni]# applies mask to CT chunk
        segments = zyx_test_segmentation(masked_sample, fit, 0, i)# segments the masked tomography
        output_voxels[i:i+ni] = segments[0] > 0.01 # appends binary resin bit-mask to the h5py file
    
    # closes all the files used after loop 
    resin_file.close()
    tomogram.close()
    implant_mask.close()
    print('Resin-mask is ready to be used')

In [28]:
def bone_mask_partial_close(sample, scale, z_fit, chunk_size):
    """Creates partially closed bone bitmask with binary_fill_holes"""
    """sample: .h5 file of the CT; scale: the scale of the data i.e 1x, 2x etc"""
    """z_fit: The distribution to segment with; chunk_size: size of CT chunks used for each iteration """
    tomogram     = h5py.File(seg_path + scale + '_' + sample[:-3]+'_concat.h5', 'r')# path to the h5py file to access data
    implant_mask = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    fit = z_fit.reshape(1,z_fit.shape[0],z_fit.shape[1])# shape the arr, so it matches the required shape for the segmentation
    
    shape = implant_mask['mask_voxels'].shape# to get dimensions to know the number of iterations through the z-axis
    bone_file = h5py.File('Bone-bitmask_' + scale + '_' + sample,'w')# create the bone bit-mask file
    bone_file.create_dataset("mask_voxels", shape, dtype=np.bool, compression="lzf")# create dataset to input data
    output_voxels = bone_file['mask_voxels']# variable to input segmented data into

    print('H5py-files are ready to be written')
    
    for i in tqdm(range(0, shape[0], chunk_size)): #shape[zyx]
        ni = min(chunk_size, shape[0]-i)#takes minimum of the chunk-size and the remaining layers to make sure all layer are included
        Layer = tomogram['voxels'][i:i+ni]# loads chunk of the CT
        masked_sample = Layer * implant_mask['mask_voxels'][i:i+ni]# applies mask to CT chunk
        segments = zyx_test_segmentation(masked_sample, fit, 0, i)# segments the masked tomography
        
        if ni %100==0:# if statement to avoid error in the last layer
            segments = np.array([ndi.binary_fill_holes(segments[0][f]>0.01) for f in range(ni)])# close holes in the segmented image
            output_voxels[i:i+ni] = segments # appends binary resin bit-mask to the h5py file
        else:
            segments = np.array([ndi.binary_fill_holes(segments[0][f]>0.01) for f in range(ni-1)])# close holes in the segmented image
            output_voxels[i:i+ni-1] = segments# appends binary resin bit-mask to the h5py file
    
    
    # closes all the files used after loop 
    bone_file.close()
    tomogram.close()
    implant_mask.close()
    print('bone-mask is ready to be used')

In [29]:
def bone_mask_fullclose(sample, scale, z_fit, chunk_size):  
    """Creates fully closed bone bitmask with convex_hull_image"""
    """sample: .h5 file of the CT; scale: the scale of the data i.e 1x, 2x etc"""
    """z_fit: The distribution to segment with; chunk_size: size of CT chunks used for each iteration """
    tomogram     = h5py.File(seg_path  + scale + '_' + sample[:-3]+'_concat.h5', 'r')# path to the h5py file to access data
    implant_mask = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    fit = z_fit.reshape(1,z_fit.shape[0],z_fit.shape[1])# shape the arr, so it matches the required shape for the segmentation
    
    # quirk of implant bit mask would make it bigger or smaller
    # The smallest are chosen to make sure the mathc in segmentation
    shape = implant_mask['mask_voxels'].shape
    shape2 = tomogram['voxels'].shape
    if shape[0] <= shape2[0]:
        shape = shape
    else:
        shape = shape2
    
    
    bone_file = h5py.File('Bone-bitmask2_' + scale + '_' + sample,'w')# create the bone bit-mask file
    bone_file.create_dataset("mask_voxels", shape, dtype=np.bool, compression="lzf")# create dataset to input data
    output_voxels = bone_file['mask_voxels']# variable to input segmented data into

    print('H5py-files are ready to be written')
    
    for i in tqdm(range(0, shape[0], chunk_size)): #shape[zyx]
        ni = min(chunk_size, shape[0]-i)#takes minimum of the chunk-size and the remaining layers to make sure all layer are included
        Layer = tomogram['voxels'][i:i+ni]# loads chunk of the CT
        masked_sample = Layer * implant_mask['mask_voxels'][i:i+ni]# applies mask to CT chunk
        #segments = zyx_test_segmentation(masked_sample, fit, 0, i)
        #segments = np.array([convex_hull_image(segments[0][f]) for f in range(ni)])
        #output_voxels[i:i+ni] = segments
        segments = zyx_stodder_segmentation(masked_sample, fit, 0, i) # only used x idx is out of range
        if ni %100==0:
            segments = np.array([convex_hull_image(segments[0][f]) for f in range(ni)])# close holes in the segmented image
            output_voxels[i:i+ni] = segments# appends binary resin bit-mask to the h5py file
        else:
            segments = np.array([convex_hull_image(segments[0][f]) for f in range(ni-1)])# closes holes in the segmented image
            output_voxels[i:i+ni-1] = segments# appends binary resin bit-mask to the h5py file
    
    # closes all the files used after loop    
    bone_file.close()
    tomogram.close()
    implant_mask.close()
    print('Bone-mask is ready to be used')

In [30]:
def final_zyx_segmentation(sample, scale, fit, zyx, chunk_size):
    """Segments CT with given probability distributions"""
    """Creates partially closed bone bitmask with binary_fill_holes"""
    """sample: .h5 file of the CT; scale: the scale of the data i.e 1x, 2x etc"""
    """fit: The distribution to segment with; chunk_size: size of CT chunks used for each iteration """
    """zyx: 0, 1 or 2 relating to Z-,Y- or X-segmentation repsectively """
    
    tomogram     = h5py.File(seg_path + scale + '_' + sample[:-3]+'_concat.h5', 'r')# path to the h5py file to access data
    implant_mask = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    bone_mask = h5py.File(mask_path + 'Bone-bitmask2_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    print('Files has been loaded')
    
    n = len(fit)# number of class
    
    # quirk of implant bit mask would make it bigger or smaller
    # The smallest are chosen to make sure the mathc in segmentation
    shape = implant_mask['mask_voxels'].shape
    shape2 = tomogram['voxels'].shape
    if shape[0] <= shape2[0]:
        shape = shape
    else:
        shape = shape2
    
    axis = choose_axis(zyx)# gives Z,Y or X as astring given the value of zyx
    
    output, files = create_h5py_files(shape, axis, scale, sample, n) # Creates h5py files
    print('H5py-files are ready to be written')
    if zyx == 0: # Z-segmentation
        for i in tqdm(range(0, shape[0], chunk_size)):
            ni = min(chunk_size, shape[0]-i)# takes minimum of the chunk-size and the remaining layers to make sure all layer are included
            
            Layer   = tomogram['voxels'][i:i+ni]# loads chunk of the CT
            implant = implant_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            bone    = bone_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            masked_sample = Layer * implant * bone# applies implant mask to CT chunk
            segments = zyx_test_segmentation(masked_sample, fit, zyx, i)# segments the masked tomography
            for seg in range(n):
                output[seg][i:i+ni] = segments[seg]# appends segmented data to compressed h5py file
    
    elif zyx == 1:# Y-segmentation
        for i in tqdm(range(0, shape[0], chunk_size)):
            ni = min(chunk_size, shape[0]-i) # takes minimum of the chunk-size and the remaining layers to make sure all layer are included
            
            Layer   = tomogram['voxels'][i:i+ni]# loads chunk of the CT
            implant = implant_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            bone    = bone_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            masked_sample = Layer * implant * bone# applies implant mask to CT chunk
            segments = zyx_test_segmentation(masked_sample, fit, zyx, 0)# segments the masked tomography
            for seg in range(n):
                output[seg][i:i+ni] = np.moveaxis(segments[seg],0,-2)# appends segmented data to compressed h5py file
                
    elif zyx == 2:# X-segmentation
        for i in tqdm(range(0, shape[0], chunk_size)): #shape[zyx]
            ni = min(chunk_size, shape[0]-i)# takes minimum of the chunk-size and the remaining layers to make sure all layer are included
            
            Layer   = tomogram['voxels'][i:i+ni]# loads chunk of the CT
            implant = implant_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            bone    = bone_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
            masked_sample = Layer * implant * bone# applies implant mask to CT chunk
            segments = zyx_test_segmentation(masked_sample, fit, zyx, 0)# segments the masked tomography
            for seg in range(n):
                output[seg][i:i+ni] = np.moveaxis(segments[seg],0,-1) # appends segmented data to compressed h5py file   

    for k in range(n): # closes the generated files again
        files[k].close()
    tomogram.close()
    implant_mask.close()
    #bone_mask.close()
    print('Files has been closed')
    print(axis.upper() + '-segmentation is done')

In [31]:
def choose_axis(dim):
    """number of dim choose the string to name the segmentation"""
    if dim == 0:
        axis = 'z'
    elif dim ==1:
        axis = 'y'
    elif dim ==2:
        axis = 'x'
    return axis

In [32]:
def rad_segmentation4(data, fit, m_ind, rmin):
    """Segments CT with given probability distributions, with the radial transformation"""
    """fit: The distribution to segment with; m_ind: mask indices; rmin: outer limit for the first chunk of layers """
    chunk = rmin# the chunk size of the inner layer
    arrs = [np.zeros_like(data).astype(float) for i in range(len(fit))]
    for i in range(len(fit[0])-chunk):# chunk is subtracted to adjust for the first layers taken in one go
        if i == 0:# to include the inner chunk first
            r1 = chunk
            hist0 = fit[:][:r1].sum(axis=(0,1))
            _,fit0 = minimizer_mask(XX,hist0,5)
            for j in range(len(fit0)):# segments the inner chunk first
                arrs[j][:,m_ind[i,0],m_ind[i,1]] += normalise(fit0[j,data[:,m_ind[i,0],m_ind[i,1]]])
        else:
            for j in range(len(fit)):
                arrs[j][:,m_ind[i,0],m_ind[i,1]] += normalise(fit[j][i+chunk,data[:,m_ind[i,0],m_ind[i,1]]])
    return arrs

In [33]:

def final_rad_segmentation(sample, scale, fit, rmin, chunk_size):
    """Segments CT wrt to radial fitted distributions"""
    """sample: .h5 file of the CT; scale: the scale of the data i.e 1x, 2x etc; rmin: outer limit for the first chunk of layers"""
    """fit: The distribution to segment with; chunk_size: size of CT chunks used for each iteration """
    tomogram     = h5py.File(seg_path + scale + '_' + sample[:-3]+'_concat.h5', 'r')# path to the h5py file to access data
    implant_mask = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    bone_mask = h5py.File(mask_path + 'Bone-bitmask2_'+ scale + '_' + sample, 'r')# path to the h5py file to access data
    print('Files has been loaded')
    mask_indices = create_radmask(tomogram['voxels'][0], rmin)# creates mask the CT
    print('Mask\'s made')
    n = len(fit)# number of classes
    
    
    # adjusting quirk
    shape = implant_mask['mask_voxels'].shape
    shape2 = tomogram['voxels'].shape
    if shape[0] <= shape2[0]:
        shape = shape
    else:
        shape = shape2
    
    output, files = create_h5py_files(shape,'rad_corrected', scale, sample, n) # Creates h5py files
    print('H5py-files are ready to be written')
    for i in tqdm(range(0, shape[0], chunk_size)):
        ni = min(chunk_size, shape[0]-i)# takes minimum of the chunk-size and the remaining layers to make sure all layer are included
        Layer = tomogram['voxels'][i:i+ni]# loads chunk of the CT
        implant = implant_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
        bone = bone_mask['mask_voxels'][i:i+ni]# loads chunk of the bit-mask
        masked_sample = Layer * implant * bone# applies implant mask to CT chunk
        segments = rad_segmentation4(masked_sample, fit, mask_indices, rmin)# segments the masked tomography
        for seg in range(n):
            output[seg][i:i+ni] = segments[seg]# appends segmented data to compressed h5py file   

    for k in range(n): # closes the generate files again
        files[k].close()
    tomogram.close()
    implant_mask.close()
    bone_mask.close()
    print('Files has been closed')
    print('Radial-segmentation is done')

In [34]:
# valid samples: Samples where volume matching worked
vs = [0, 2, 4, 6, 10, 13, 15, 17]

[0, 2, 4, 6, 10, 13, 15, 17]

In [30]:
# THIS IS WHERE THE WHOLE PROCESS STARTS
# 1st Step: Scans sample-data converts into four 2D histograms - radial, z, y and x
# Histograms are saved

scale = '1x' # scaling 
N = vs[1]

mak1 = h5py.File(mask_path + 'Implant-bitmask_'+ scale + '_' + samples_1x[N], 'r')
mak2 = h5py.File(mask_path + 'Bone-bitmask2_'  + scale + '_' + samples_1x[N], 'r')

masks = mak1,mak2

load_n_save_hists(samples_1x[N], '1x', masks, False, masked = 2, concat = True)


100%|██████████| 17/17 [07:43<00:00, 27.29s/it]

Data has been loaded!


In [65]:
# 3rd Step: Fits a pdf to each peak of each layer
the_fits = combined_autofitter(data,5)

  0%|          | 0/868 [00:00<?, ?it/s]/home/madsapp/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/madsapp/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 868/868 [03:32<00:00,  4.09it/s] 

The y-histogram has been fitted


In [92]:
# 4th Step: THe fitting process has some flaws, this step tries to correct them and fill in the lacking information
# After testing, this step seems more or less redundant, skip to step 5 

In [93]:
# 5th Step: Find ridges of the 2D fits to separate distributions into classes

In [66]:
rad_labels = find_ridges(rad_fits, label_frac = 8, e_iter = 1, d_iter = 20, show = 1)

In [195]:
z_labels = find_ridges(z_fits, label_frac = 5, e_iter = 1, d_iter = 10, show = 1)

In [73]:
y_labels   = find_ridges(y_fits, label_frac = 5, e_iter = , d_iter = 10, show = 1)

In [58]:
x_labels   = find_ridges(the_fits[3], label_frac = 8, d_iter = 10, show = 1)

In [109]:
# Saving all labels 
save_labels(rad_labels, 'rad', samples_1x[N],scale, 0)
save_labels(z_labels,   'z'  , samples_1x[N],scale, 0)
save_labels(y_labels,   'y'  , samples_1x[N],scale, 0)
save_labels(x_labels,   'x'  , samples_1x[N],scale, 0)

In [110]:
#Step 6.1: Sorts distributions into classes
sorted_rad = distr_sorter(the_fits[0][0], rad_labels)
sorted_z =   distr_sorter(the_fits[1][0], z_labels)
sorted_y =   distr_sorter(the_fits[2][0], y_labels)
sorted_x =   distr_sorter(the_fits[3][0], x_labels)
print(11 * '====')
#Step 6.2: Sorts parameters into classes
sorted_varrad = parameter_sorter(the_fits[0][1], rad_labels)
sorted_varz =   parameter_sorter(the_fits[1][1], z_labels)
sorted_vary =   parameter_sorter(the_fits[2][1], y_labels)
sorted_varx =   parameter_sorter(the_fits[3][1], x_labels)
print(11 * '====')
# Step 6.3: combines distributions with respective parameters
comb_distr  = sorted_rad,    sorted_z,    sorted_y,    sorted_x
comb_params = sorted_varrad, sorted_varz, sorted_vary, sorted_varx

Shape of array: (5, 433, 254)
Shape of array: (4, 805, 254)
Shape of array: (5, 868, 254)
Shape of array: (5, 868, 254)
Size of array 433 - Number of labels 5
Size of array 805 - Number of labels 4
Size of array 868 - Number of labels 5
Size of array 868 - Number of labels 5


In [ ]:
# saves fitted distributions and parameters for later use
save_fits_rzyx(comb_distr, comb_params, '1x', samples_1x[N], 2)

In [57]:
# If crash because lack of memory - reload fits and decrease chunk_size
#sorted_rad, sorted_z, sorted_y, sorted_x = load_fits(samples_1x[N], '1x')

In [35]:
chunk_size = 100 # number of layer to chug on at a time
scale = '1x'

In [60]:
# Radial segmentation
final_rad_segmentation(samples_1x[N], scale, sorted_rad, 20, chunk_size)

In [78]:
# Z-segmentation
final_zyx_segmentation(samples_1x[N], scale, sorted_z, 0, chunk_size)

In [80]:
# Y-segmentation
final_zyx_segmentation(samples_1x[N], scale, sorted_y, 1, chunk_size)

  0%|          | 0/9 [00:00<?, ?it/s]

Files has been loaded
H5py-files are ready to be written


100%|██████████| 9/9 [00:43<00:00,  4.81s/it]

Files has been closed
Y-segmentation is done


In [ ]:
# X-segmentation
final_zyx_segmentation(samples_1x[N], scale, sorted_x, 2, chunk_size)

In [340]:
# now we are done! yay!